In [51]:
!pip install -e git+https://github.com/Mottl/GetOldTweets3#egg=GetOldTweets3

Defaulting to user installation because normal site-packages is not writeable
Obtaining GetOldTweets3 from git+https://github.com/Mottl/GetOldTweets3#egg=GetOldTweets3
  Cloning https://github.com/Mottl/GetOldTweets3 to ./src/getoldtweets3
  Running command git clone -q https://github.com/Mottl/GetOldTweets3 /home/aravind/Desktop/Maha/COVID19/src/getoldtweets3
  Running setup.py develop for GetOldTweets3


In [1]:
import tweepy
import pandas as pd
import json
import csv
from textblob import TextBlob
import re
import nltk
from  geopy.geocoders import Nominatim
from datetime import datetime
import pycountry
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/aravind/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import GetOldTweets3 as got
import pickle
import os
import pandas as pd

In [39]:
def extract_tweets(str_strt_date,str_end_date,lst_search_string,filename):
    search_string=' '.join([str(elem) for elem in lst_search_string])

    tweetCriteria = got.manager.TweetCriteria()\
                        .setQuerySearch(search_string)\
                        .setSince(str_strt_date)\
                        .setUntil(str_end_date)\
                        .setLang('en')

    result= got.manager.TweetManager.getTweets(tweetCriteria)
    pickle.dump( result,open(filename, "wb" ) )
    print("Done extracting tweets")

In [4]:
def remove_duplicate(data_set):
    return data_set.drop_duplicates(subset =["id","username"],keep = False) 

In [41]:
def transform_tweets(file, save_file):
    results= pickle.load( open(file, "rb" ) )
    
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"],index=None)
    data_set["username"] = [tweet.username for tweet in results]
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.date for tweet in results]
    #extracting the date from the created at column and making a date column
    data_set['date'] =  pd.to_datetime(data_set['created_at']).dt.date
    data_set["retweets"] = [tweet.retweets for tweet in results]
    data_set["hashtags"] = [tweet.hashtags for tweet in results]
    data_set["geo"] = [tweet.geo for tweet in results]
    data_set=remove_duplicate(data_set)
    
    polarity_scores = []
    subjectivity_scores = []
    for tweet in results:
        tweets = tweet.text
        tweets = ' '.join(re.sub("(@[A-Za-z0-9]+) | ({*0-9A-Za-z \t]) |] (\wt:\/\/\St+)", " ", tweets).split())
        tweets = ' '.join(re.sub('RT',' ', tweets).split())  
        
        blob = TextBlob(tweets.strip())
        #global polarity 
        #global subjectivity 

        polarity = 0
        subjectivity = 0

        for sent in blob.sentences:
            polarity += round(sent.sentiment.polarity,2)
            subjectivity += round(sent.sentiment.subjectivity,2)
        polarity = polarity/len(blob.sentences)
        subjectivity = subjectivity/len(blob.sentences)
        
        polarity_scores.append(polarity)
        subjectivity_scores.append(subjectivity)
    
    data_set["polarity"] = polarity_scores
    data_set["subjectivity"] = subjectivity_scores

    if os.path.exists(save_file):
        df=pd.read_csv(save_file)
        df_2=pd.concat([data_set,df])
        df_2=remove_duplicate(df_2)

        df_2.to_csv(save_file)

    else:
        data_set.to_csv(save_file)

    print("Done tranforming tweets \nShape:{}".format(data_set.shape))

In [28]:
extract_tweets("2020-03-01","2020-03-05",['corona OR corona19 OR Dow OR Nasdaq OR S&P','USA'],"raw_tweets_mar_1_5.pkl")

Done extracting tweets


In [42]:
transform_tweets("raw_tweets_mar_1_5.pkl","market_mar_1_5_norm.csv")

Done tranforming tweets 
Shape:(2721, 10)


In [40]:
extract_tweets("2020-03-23","2020-03-28",['corona OR corona19 OR DJI OR Nasdaq OR S&P', 'USA'],"raw_tweets_count_mar_23_28.pkl")
transform_tweets("raw_tweets_count_mar_23_28.pkl","tweets_count_mar_23_28.csv")

Done extracting tweets
Done tranforming tweets 
Shape:(7285, 10)


In [43]:
transform_tweets("raw_tweets_count_mar_23_28.pkl","tweets_count_mar_23_28_norm.csv")

Done tranforming tweets 
Shape:(7285, 10)


In [30]:
extract_tweets("2020-03-17","2020-03-21",['corona OR corona19 OR Dow OR Nasdaq OR S&P','USA'],"raw_tweets_mar_17_22.pkl")

Done extracting tweets


In [31]:
transform_tweets("raw_tweets_mar_17_22.pkl","market_mar_17_21.csv")

Done tranforming tweets 
Shape:(8730, 10)


In [35]:
recovery = pd.read_csv('market_mar_17_21.csv')
recovery['polarity'].value_counts()

 0.00    4874
 0.50     192
 0.20     136
 0.10     117
-0.12     103
         ... 
-0.95       1
 0.67       1
 0.84       1
-1.39       1
 0.68       1
Name: polarity, Length: 392, dtype: int64

In [37]:
recovery['subjectivity'].value_counts()

0.00    4563
1.00     295
0.50     271
0.40     152
0.60     127
        ... 
1.91       1
2.57       1
1.78       1
4.15       1
2.45       1
Name: subjectivity, Length: 314, dtype: int64

In [36]:
ill = pd.read_csv('market_mar_1_5.csv')
ill['polarity'].value_counts()

 0.00    1277
 0.50      69
 0.10      49
 0.20      44
-0.20      44
         ... 
 0.05       1
 1.20       1
-2.18       1
 1.02       1
 1.06       1
Name: polarity, Length: 294, dtype: int64

In [38]:
ill['subjectivity'].value_counts()

0.00    1161
0.50      98
1.00      85
0.40      54
0.60      44
        ... 
3.05       1
0.59       1
1.18       1
1.57       1
1.06       1
Name: subjectivity, Length: 274, dtype: int64